In [1]:
import importlib
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import datetime as dt
from datetime import datetime
from datetime import timedelta
from datetime import date

import PrjtCF_module as cf
from PrjtCF_module.account import Account

In [30]:
from PrjtCF_module import account as acc

In [31]:
acc.Account

PrjtCF_module.account.Account

In [34]:
import importlib
importlib.reload(acc)

ModuleNotFoundError: No module named 'genfunc'

In [51]:
importlib.reload(cf)

<module 'PrjtCF_module' from '/Users/KP_Hong/Documents/PyPrjt/[lib]PrjtCashFlowModel_v5/PrjtCF_module/__init__.py'>

# Input Basic Data

In [2]:
prd_prjt = 24 #개월
prd_loan = 20 #개월
idx_prjt = cf.Index('2021-01', periods=(prd_prjt+1), freq='M')
idx_loan = cf.Index('2021-02', periods=(prd_loan+1), freq='M')

# Input Sales Data

In [3]:
# Initial Sales Data
# Product A : 상온창고
prdtA = cf.Account(idx_prjt, "ProductA", ("Prdt", "A"))
prdtA.rent = 21_000 # 월 임대료 21,000원/평 가정
prdtA.area = 8_000 # 면적 8,000평
prdtA.cap = 0.053 # Cap rate 5.3%
prdtA.amt = prdtA.rent * 12 * prdtA.area / prdtA.cap / 1_000_000
prdtA.addamt(prdtA.index[0], prdtA.amt)
prdtA.subscdd(prdtA.index[21], prdtA.amt)

# Initial Sales Data
# Product B : 저온창고
prdtB = cf.Account(idx_prjt, "ProductB", ("Prdt", "B"))
prdtB.rent = 50_000 # 월 임대료 50,000원/평 가정
prdtB.area = 6_500 # 면적 6,500평
prdtB.cap = 0.058 # Cap rate 5.8%
prdtB.amt = prdtB.rent * 12 * prdtB.area / prdtB.cap / 1_000_000
prdtB.addamt(prdtB.index[0], prdtA.amt)
prdtB.subscdd(prdtB.index[21], prdtB.amt)

# Merge
sales = cf.Merge({'prdtA':prdtA, 'prdtB':prdtB})

# Input Cost Data

In [4]:
dct_cost = {}

# 공정률
idx_prjt.prcs_rate = Series(np.ones(len(idx_prjt)) / len(idx_prjt), 
                       index=idx_prjt.index)

# Initial Cost Data
cstlnd = cf.Account(idx_prjt, "Cost_Land", ("Cost", "Land"))
cstlnd.amt = 15_500.000 # 최초 1회 지급
cstlnd.addscdd(idx_prjt.index[0], cstlnd.amt)
dct_cost['cstlnd'] = cstlnd

cstcstrn = cf.Account(idx_prjt, "Cost_Construction", ("Cost", "Construction"))
cstcstrn.amt = 50_000.000 # 공사비(지급) : 공정률에 따라 지급
cstcstrn.addscdd(idx_prjt.index, cstcstrn.amt * idx_prjt.prcs_rate)
dct_cost['cstcstrn'] = cstcstrn

cstcstrnb = cf.Account(idx_prjt, "Cost_Construction B", ("Cost", "ConstructionB"))
cstcstrnb.amt = 3_000.000 # 최초 1회 지급
cstcstrnb.addscdd(idx_prjt.index[0], cstcstrnb.amt)
dct_cost['cstcstrnb'] = cstcstrnb

cstmrktg = cf.Account(idx_prjt, "Cost_Marketing", ("Cost", "Marketing"))
cstmrktg.amt = 2_000.000 # 공정률에 따라 지급
cstmrktg.addscdd(idx_prjt.index, cstmrktg.amt * idx_prjt.prcs_rate)
dct_cost['cstmrktg'] = cstmrktg

csttax = cf.Account(idx_prjt, "Cost_Tax", ("Cost", "Tax"))
csttax.amt = 3_000.000 # 준공시 1회 지급
csttax.addscdd(idx_prjt.index[11], csttax.amt)
dct_cost['csttax'] = csttax

cstoprtg = cf.Account(idx_prjt, "Cost_Operating", ("Cost", "Operating"))
cstoprtg.amt = 2_200.000
cstoprtg.addscdd(idx_prjt.index, cstoprtg.amt * idx_prjt.prcs_rate)
dct_cost['cstoprtg'] = cstoprtg

cost = cf.Merge(dct_cost)

# Input Loan Data

In [24]:
dct_loan = {}

tra = cf.Loan(idx_loan, amt_ntnl=60_000, rate_fee=0.015, 
             rate_IR=0.05, title="Tr.A", tag=("loan", "A"))
dct_loan['tra'] = tra

trb = cf.Loan(idx_loan, amt_ntnl=20_000, rate_fee=0.03, 
             rate_IR=0.07, title="Tr.B", tag=("loan", "B"))
dct_loan['trb'] = trb

loan = cf.Merge(dct_loan)

# Cash Flow

In [ ]:
# Make accounts
acc_oprtg = cf.Account(idx_prjt, "Operating Account", ("Account", "Operating"))
